In [1]:
import pandas as pd
import fba_utils
from fba_utils import get_flux_bounds, convert_transcriptomics_to_enzyme_activity
import cobra
import numpy as np
import os 

In [2]:
os.getcwd()

'/home/connah/Documents/Repos/syn_bmca'

## 1) collect input

In [3]:
# get the cobra model for S. elongatus
syn_model = cobra.io.read_sbml_model('models/syn_elong.xml')
[m.id for m in syn_model.metabolites if m.id.startswith('sucr') ]

Restricted license - for non-production use only - expires 2025-11-24


['sucr_c', 'sucr_e']

In [5]:
# get the fluxomics data without the photon contraint
fluxomic_wo_photon_df = pd.read_excel('data/GAPDH-fluxomics/raw_data/1-s2.0-S1096717623001222-mmc5.xlsx',sheet_name=0,index_col=1, skiprows=1, header=0)
fluxomic_wo_photon_df.index.name = 'Reaction_ID'
display(fluxomic_wo_photon_df)

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,MaxBiomass,minFlux,maxFlux,lb,ub,maxFlux-minFlux,...,lb.1,ub.1,maxFlux-minFlux.1,Unnamed: 18,MaxBiomass.2,minFlux.2,maxFlux.2,lb.2,ub.2,maxFlux-minFlux.2
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
EX_gln__L_e,NaN,gln__L.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,1000.0,0.000000e+00,...,0.0000,1000.0,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.0000,1000.0000,0.000000e+00
EX_hco3_e,NaN,hco3.e <->,NaN,NaN,-1.990000,-1.990000,-0.553000,-1.9900,1000.0,1.437000e+00,...,-1.9900,1000.0,1.520034e+00,NaN,-1.990000,-1.990000,-0.749966,-1.9900,1000.0000,1.240034e+00
EX_mn2_e,NaN,mn2.e <->,NaN,NaN,0.000000,0.000000,0.000000,-0.0009,1000.0,0.000000e+00,...,-0.0009,1000.0,0.000000e+00,NaN,0.000000,0.000000,0.000000,-0.0009,1000.0000,0.000000e+00
EX_arg__L_e,NaN,arg__L.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,1000.0,0.000000e+00,...,0.0000,1000.0,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.0000,1000.0000,0.000000e+00
ADPT,NaN,ade.c + prpp.c <-> amp.c + ppi.c,NaN,NaN,0.000095,0.000095,0.000095,-1000.0000,1000.0,5.587498e-12,...,-1000.0000,1000.0,1.587829e-09,NaN,0.000073,0.000073,0.000073,-1000.0000,1000.0000,1.474248e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OXGDC,NaN,akg.c + h.c <-> co2.c + sucsal.c,NaN,NaN,-0.087086,-0.087086,159.544334,-1000.0000,1000.0,1.596314e+02,...,-1000.0000,1000.0,1.597635e+02,NaN,-0.064113,-0.064118,159.555529,-1000.0000,1000.0000,1.596196e+02
SSALy,NaN,h2o.c + nadp.c + sucsal.c <-> 2 h.c + nadph.c ...,NaN,NaN,-0.087086,-0.087086,159.544334,-1000.0000,1000.0,1.596314e+02,...,-1000.0000,1000.0,1.597635e+02,NaN,-0.064113,-0.064118,159.555529,-1000.0000,1000.0000,1.596196e+02
SK_Sucrose,NaN,sucr.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,0.0,0.000000e+00,...,0.0000,0.0,0.000000e+00,NaN,0.080200,0.080200,0.080200,0.0802,0.0802,0.000000e+00


In [6]:
# get the fluxomics data with the photon contraint
fluxomic_photon_df = pd.read_excel('data/GAPDH-fluxomics/raw_data/1-s2.0-S1096717623001222-mmc5.xlsx',sheet_name=1,index_col=1, skiprows=1, header=0)
fluxomic_photon_df.index.name = 'Reaction_ID'
display(fluxomic_photon_df)

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,MaxBiomass,minFlux,maxFlux,lb,ub,maxFlux-minFlux,...,lb.4,ub.4,maxFlux-minFlux.4,Unnamed: 39,MaxBiomass.5,minFlux.5,maxFlux.5,lb.5,ub.5,maxFlux-minFlux.5
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
EX_gln__L_e,NaN,gln__L.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,1000.0,0.000000e+00,...,0.0000,1000.0000,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.0000,1000.0000,0.000000e+00
EX_hco3_e,NaN,hco3.e <->,NaN,NaN,-1.990000,-1.990000,-0.553000,-1.9900,1000.0,1.437000e+00,...,-1.9900,1000.0000,1.240034e+00,NaN,-1.990000,-1.990000,-0.653759,-1.9900,1000.0000,1.336241e+00
EX_mn2_e,NaN,mn2.e <->,NaN,NaN,0.000000,0.000000,0.000000,-0.0009,1000.0,0.000000e+00,...,-0.0009,1000.0000,0.000000e+00,NaN,0.000000,0.000000,0.000000,-0.0009,1000.0000,0.000000e+00
EX_arg__L_e,NaN,arg__L.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,1000.0,0.000000e+00,...,0.0000,1000.0000,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.0000,1000.0000,0.000000e+00
ADPT,NaN,ade.c + prpp.c <-> amp.c + ppi.c,NaN,NaN,0.000095,0.000095,0.000095,-1000.0000,1000.0,5.587498e-12,...,-1000.0000,1000.0000,1.474248e-09,NaN,0.000069,0.000069,0.000069,-1000.0000,1000.0000,1.787061e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OXGDC,NaN,akg.c + h.c <-> co2.c + sucsal.c,NaN,NaN,-0.087086,-0.087086,16.377065,-1000.0000,1000.0,1.646415e+01,...,-1000.0000,1000.0000,1.656238e+01,NaN,0.001719,0.001712,0.023948,-1000.0000,1000.0000,2.223595e-02
SSALy,NaN,h2o.c + nadp.c + sucsal.c <-> 2 h.c + nadph.c ...,NaN,NaN,-0.087086,-0.087086,16.377065,-1000.0000,1000.0,1.646415e+01,...,-1000.0000,1000.0000,1.656238e+01,NaN,0.001719,0.001712,0.023948,-1000.0000,1000.0000,2.223595e-02
SK_Sucrose,NaN,sucr.e ->,NaN,NaN,0.000000,0.000000,0.000000,0.0000,0.0,0.000000e+00,...,0.0802,0.0802,0.000000e+00,NaN,0.080200,0.080200,0.080200,0.0802,0.0802,0.000000e+00


## 2) Relabel index through reaction IDs

In [7]:
# manual curation 

# dict[original ID] = new ID
mapped_reaction_IDs={}
# mapped_reaction_IDs['Biomass_acgam_acmum'] = 'BIOMASS_ACGAM_ACMUM'
mapped_reaction_IDs['GLUDGE_PALM_PALM'] = 'GLUDGE_OLE_PALM'
mapped_reaction_IDs['GLUDGS_PALM_PALM'] = 'GLUDGS_OLE_PALM'
mapped_reaction_IDs['DGDGS_PALM_PALM'] = 'DGDGS_PALM_PALM_c'
mapped_reaction_IDs['SK_colipacy_e'] = 'EK_colipacy_e'
mapped_reaction_IDs['CDPDAGS_HDE_PALM'] = 'CDPDAGS_HDE_PALM_c'
mapped_reaction_IDs['CDPDAGS_PALM_PALM'] = 'CDPDAGS_PALM_PALM_c'
mapped_reaction_IDs['PGPS_HDE_PALM'] = 'PGPS_HDE_PALM_h'
mapped_reaction_IDs['PGPP_HDE_PALM'] = 'PGPP_HDE_PALM_h'
mapped_reaction_IDs['PGPS_PALM_PALM'] = 'PGPS_PALM_PALM_h'
mapped_reaction_IDs['PGS_PALM_PALM'] = 'PGPP160'
mapped_reaction_IDs['SK_Sucrose'] = 'EX_sucr_e'


fluxomic_wo_photon_df = fluxomic_wo_photon_df.rename(index=mapped_reaction_IDs)
fluxomic_photon_df = fluxomic_photon_df.rename(index=mapped_reaction_IDs)



drop_indexes = ['bm_glucan','BIOMASS_CROTENOIDS','Biomass_acgam_acmum','BIOMASS_LIPIDS']

fluxomic_wo_photon_df = fluxomic_wo_photon_df.drop(drop_indexes)
fluxomic_photon_df = fluxomic_photon_df.drop(drop_indexes)



In [8]:
# display reactions not recognised by the input cobra model
for ind in fluxomic_wo_photon_df.index:
    if ind not in syn_model.reactions:
        print(ind)
        print(fluxomic_wo_photon_df.loc[ind,'Unnamed: 2'])
        print()



DGDGS_PALM_PALM_c
udpgal.c + mgdg160.c  -> udp.c + h.c + dgdg160.c 

EK_colipacy_e
colipacy1.e -> 

CDPDAGS_HDE_PALM_c
h.c + ctp.c + pa1619Z160.c  -> ppi.c + cdp12dgr1619Z160.c 

PGPS_HDE_PALM_h
glyc3p.c + cdp12dgr1619Z160.c  -> h.c + cmp.c + pgp1619Z160.c 

PGPP_HDE_PALM_h
pgp1619Z160.c -> pg1619Z160.c

CDPDAGS_PALM_PALM_c
h.c + ctp.c + pa160.c  -> ppi.c + cdp12dgr160.c 

PGPS_PALM_PALM_h
glyc3p.c + cdp12dgr160.c  -> h.c + cmp.c + pgp160.c 

PGPP160
pgp160.c -> pg160.c

OXGDC
akg.c + h.c <-> co2.c + sucsal.c

SSALy
h2o.c + nadp.c + sucsal.c <-> 2 h.c + nadph.c + succ.c

CPMPS_1
38ch2gtp[c] -> cpmp[c] + ppi[c]



## 3) Split and relabel DataFrame columns

In [9]:
fluxomic_wo_photon_df = fluxomic_wo_photon_df.drop([col for col in fluxomic_wo_photon_df.columns if 'Unnamed' in col], axis=1)
fluxomic_wo_photon_df = fluxomic_wo_photon_df.drop([col for col in fluxomic_wo_photon_df.columns if 'lb' in col], axis=1)
fluxomic_wo_photon_df = fluxomic_wo_photon_df.drop([col for col in fluxomic_wo_photon_df.columns if 'ub' in col], axis=1)
fluxomic_wo_photon_df = fluxomic_wo_photon_df.drop([col for col in fluxomic_wo_photon_df.columns if 'maxFlux-minFlux' in col], axis=1)

new_column_names = ['MaxBiomass','minFlux','maxFlux']

fluxomic_wo_photon_WT_Con_df = fluxomic_wo_photon_df[[col for col in fluxomic_wo_photon_df.columns if '.' not in col]]
fluxomic_wo_photon_WT_NaCl_Con_df = fluxomic_wo_photon_df[[col for col in fluxomic_wo_photon_df.columns if '.1' in col]]
fluxomic_wo_photon_WT_NaCl_Con_df.columns = new_column_names
fluxomic_wo_photon_cscB_NaCl_Con_df = fluxomic_wo_photon_df[[col for col in fluxomic_wo_photon_df.columns if '.2' in col]]


display(fluxomic_wo_photon_WT_Con_df)
display(fluxomic_wo_photon_WT_NaCl_Con_df)
display(fluxomic_wo_photon_cscB_NaCl_Con_df)

,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.553000
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000095,0.000095,0.000095
...,...,...,...
PGPP160,0.000183,0.000183,0.000183
OXGDC,-0.087086,-0.087086,159.544334
SSALy,-0.087086,-0.087086,159.544334


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.469966
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000099,0.000099,0.000099
...,...,...,...
PGPP160,0.000190,0.000190,0.000190
OXGDC,-0.079827,-0.079832,159.683662
SSALy,-0.079827,-0.079832,159.683662


,MaxBiomass.2,minFlux.2,maxFlux.2
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.749966
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000073,0.000073,0.000073
...,...,...,...
PGPP160,0.000009,0.000009,0.000009
OXGDC,-0.064113,-0.064118,159.555529
SSALy,-0.064113,-0.064118,159.555529


In [10]:
fluxomic_photon_df = fluxomic_photon_df.drop([col for col in fluxomic_photon_df.columns if 'Unnamed' in col], axis=1)
fluxomic_photon_df = fluxomic_photon_df.drop([col for col in fluxomic_photon_df.columns if 'lb' in col], axis=1)
fluxomic_photon_df = fluxomic_photon_df.drop([col for col in fluxomic_photon_df.columns if 'ub' in col], axis=1)
fluxomic_photon_df = fluxomic_photon_df.drop([col for col in fluxomic_photon_df.columns if 'maxFlux-minFlux' in col], axis=1)

new_column_names = ['MaxBiomass','minFlux','maxFlux']

fluxomic_photon_WT_ConP_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.' not in col]]
fluxomic_photon_WT_ConPMFA_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.1' in col]]
fluxomic_photon_WT_ConPMFA_df.columns = new_column_names
fluxomic_photon_WT_NaCl_ConP_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.2' in col]]
fluxomic_photon_WT_NaCl_ConP_df.columns = new_column_names

fluxomic_photon_WT_NaCl_ConPMFA_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.3' in col]]
fluxomic_photon_WT_NaCl_ConPMFA_df.columns = new_column_names
fluxomic_photon_cscB_ConP_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.4' in col]]
fluxomic_photon_cscB_ConP_df.columns = new_column_names
fluxomic_photon_cscB_ConPMFA_df = fluxomic_photon_df[[col for col in fluxomic_photon_df.columns if '.5' in col]]
fluxomic_photon_cscB_ConPMFA_df.columns = new_column_names


display(fluxomic_photon_WT_ConP_df)
display(fluxomic_photon_WT_ConPMFA_df)
display(fluxomic_photon_WT_NaCl_ConP_df)
display(fluxomic_photon_WT_NaCl_ConPMFA_df)
display(fluxomic_photon_cscB_ConP_df)
display(fluxomic_photon_cscB_ConPMFA_df)


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.553000
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000095,0.000095,0.000095
...,...,...,...
PGPP160,0.000183,0.000183,0.000183
OXGDC,-0.087086,-0.087086,16.377065
SSALy,-0.087086,-0.087086,16.377065


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.553000
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000095,0.000095,0.000095
...,...,...,...
PGPP160,0.000183,0.000183,0.000183
OXGDC,-0.010786,-0.010786,0.047910
SSALy,-0.010786,-0.010786,0.047910


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.469966
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000099,0.000099,0.000099
...,...,...,...
PGPP160,0.000190,0.000190,0.000190
OXGDC,-0.079827,-0.079832,16.569463
SSALy,-0.079827,-0.079832,16.569463


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.293471
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000091,0.000091,0.000091
...,...,...,...
PGPP160,0.000174,0.000174,0.000174
OXGDC,0.013312,0.013305,0.068407
SSALy,0.013312,0.013305,0.068407


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.749966
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000073,0.000073,0.000073
...,...,...,...
PGPP160,0.000009,0.000009,0.000009
OXGDC,-0.064113,-0.064118,16.498264
SSALy,-0.064113,-0.064118,16.498264


,MaxBiomass,minFlux,maxFlux
Reaction_ID,,,
EX_gln__L_e,0.000000,0.000000,0.000000
EX_hco3_e,-1.990000,-1.990000,-0.653759
EX_mn2_e,0.000000,0.000000,0.000000
EX_arg__L_e,0.000000,0.000000,0.000000
ADPT,0.000069,0.000069,0.000069
...,...,...,...
PGPP160,0.000009,0.000009,0.000009
OXGDC,0.001719,0.001712,0.023948
SSALy,0.001719,0.001712,0.023948


## 4) Save to processed data

In [11]:
fluxomic_wo_photon_WT_Con_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_wo_photon_WT_Con.csv')
fluxomic_wo_photon_WT_NaCl_Con_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_wo_photon_WT_NaCl_Con.csv')
fluxomic_wo_photon_cscB_NaCl_Con_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_wo_photon_cscB_NaCl_Con.csv')
fluxomic_photon_WT_ConP_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_WT_ConP.csv')
fluxomic_photon_WT_ConPMFA_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_WT_ConPMFA.csv')
fluxomic_photon_WT_NaCl_ConP_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_WT_NaCl_ConP.csv')
fluxomic_photon_WT_NaCl_ConPMFA_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_WT_NaCl_ConPMFA.csv')
fluxomic_photon_cscB_ConP_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_cscB_ConP.csv')
fluxomic_photon_cscB_ConPMFA_df.to_csv('data/GAPDH-fluxomics/processed_data/fluxomic_photon_cscB_ConPMFA.csv')